In [55]:
#load data

import os                    #python中的os模块用于处理文件和目录
import skimage               #python中的skimage模块用于图像处理
import numpy as np           #python中的numpy模块用于科学计算
from skimage import data,transform
from skimage import img_as_float
from skimage.color import rgb2gray     #rgb2gray将图片转化为灰度
import matplotlib.pyplot as plt
import random       #随机
from PIL import Image


def load_data(data_directory):
    directories=[d for d in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory,d))]
    #d is every classification file
    labels=[]
    images=[]
    names=[]
    for d in directories:
        #每一类的路径
        label_directory=os.path.join(data_directory,d)
        file_names=[os.path.join(label_directory,f) for f in os.listdir(label_directory) if f.endswith(".jpeg") or f.endswith('.JPG') ]
        for f in file_names:

            img=skimage.data.imread(f, as_grey='True')
#             img[img>color] = 1
            images.append(img)  #read image
            labels.append(d)                   #read label
            names.append(f)
    return images,labels,names

#images and labels are list

ROOT_PATH="/Users/chenlingna/Desktop/Project/MelbourneCityTraining/StreetViewDownload/TensorFlowTrain"
color = 150/255
#data_directory="/Users/chenlingna/Desktop/Project/StreetViewDownload/TensorFlowTrain/Training"
# train_data_directory=os.path.join(ROOT_PATH,"Training")
# test_data_directory=os.path.join(ROOT_PATH,"Testing")
data_directory=os.path.join(ROOT_PATH,"Training")



In [56]:
#Load Data Set
images_Total,labels_Total,names_Total=load_data(data_directory)


In [57]:
from sklearn.model_selection import train_test_split
images, test_images, labels, test_labels, names, test_names  = train_test_split(images_Total, labels_Total, names_Total, test_size=0.1, random_state=90051)

In [58]:
#Load Training Set
# images,labels,names=load_data(train_data_directory)

images28=[]
for img in images:
#     img[img>color]=1
#     img[img<color]=0
    images28.append(img)
    
# Rescale the images in the `images` array
images28 = [transform.resize(image, (28, 28)) for image in images28]
# Convert `images28` to an array
images28 = np.array(images28)
# # Convert `images` to grayscale
# images28 = rgb2gray(images28)





/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [59]:
#Load Testing Set
# test_images, test_labels, test_names = load_data(test_data_directory)

test_images28=[]
for img in test_images:
#     img[img>color]=1
#     img[img<color]=0
    test_images28.append(img)

# Transform the images to 28 by 28 pixels
test_images28 = [transform.resize(image, (28, 28)) for image in test_images28]
test_images28 = np.array(test_images28)
# # Convert to grayscale
# from skimage.color import rgb2gray

# test_images28 = rgb2gray(np.array(test_images28))


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [60]:
class DatasetIterator:
    """
    An iterator that returns randomized batches from a data set (with features and labels)
    """
    def __init__(self, features, labels, batch_size):
        assert(features.shape[0]==labels.shape[0])
        assert(batch_size > 0 and batch_size <= features.shape[0])
        self.features = features
        self.labels = labels
        self.num_instances = features.shape[0]
        self.batch_size = batch_size
        self.num_batches = self.num_instances//self.batch_size
        if (self.num_instances%self.batch_size!=0):
            self.num_batches += 1
        self._i = 0
        self._rand_ids = None

    def __iter__(self):
        self._i = 0
        self._rand_ids = np.random.permutation(self.num_instances)
        return self
    
    def next(self):
        self.__next__(self)
    
    def __next__(self):
        if self.num_instances - self._i >= self.batch_size:
            this_rand_ids = self._rand_ids[self._i:self._i + self.batch_size]
            self._i += self.batch_size
            return self.features[this_rand_ids], self.labels[this_rand_ids]
        elif self.num_instances - self._i > 0:
            this_rand_ids = self._rand_ids[self._i::]
            self._i = self.num_instances
            return self.features[this_rand_ids], self.labels[this_rand_ids]
        else:
            raise StopIteration()
            
batch_size = 100
labels=np.array(labels)
train_iterator = DatasetIterator(images28, labels, batch_size)

In [61]:
#CNN
import tensorflow as tf
IM_WIDTH = images28.shape[1]      # width of an image in pixels
IM_HEIGHT = images28.shape[2] 
NUM_CLASSES = 2
tf.reset_default_graph()
with tf.variable_scope('input'):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28], name='image')
    Y = tf.placeholder(tf.int32, [None], name='label')
    
DEPTH_C1 = 8       # depth of convolutional layer #1
DEPTH_C2 = 16      # depth of convolutional layer #2
UNITS_D1 = 256     # number of neurons in dense layer #1
with tf.variable_scope('cnn_model'):
    # Boolean placeholder which is set to True for training, and False for inference.
    # This is required to implement dropout. 
    training_mode = tf.placeholder(dtype=tf.bool, name='training_mode')
    
    # Input Layer
    input_layer = tf.reshape(X, [-1, IM_WIDTH, IM_HEIGHT, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=DEPTH_C1, kernel_size=[5, 5], 
                             padding='same', activation=tf.nn.relu, use_bias=True, 
                             name='conv_layer_1')

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, 
                                    name='pool_layer_1')

    # Convolutional Layer #2 
    conv2 = tf.layers.conv2d(inputs=pool1, filters=DEPTH_C2, kernel_size=[5, 5], 
                             padding='same', activation=tf.nn.relu, use_bias=True, 
                             name='conv_layer_2') # fill in

    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, 
                                    name='pool_layer_2') # fill in

    # Dense Layer #1
    pool2_flat = tf.reshape(pool2, shape=[-1, 7*7*DEPTH_C2], name='pool_layer_2_flat')
    dense = tf.layers.dense(inputs=pool2_flat, units=UNITS_D1, activation=tf.nn.relu, 
                            name='dense_layer_1')
    dropout = tf.layers.dropout(inputs=dense, rate=0.7, training=training_mode, name='dropout')

    # Dense Layer #2 (Logits Layer)
    logits = tf.layers.dense(inputs=dropout, units=NUM_CLASSES, use_bias=True,
                             name='dense_layer_2')
    
    # Predicted labels
    predictions = tf.argmax(logits, axis=1)

In [62]:
with tf.variable_scope('loss'):
    loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
with tf.variable_scope('train'):
    opt = tf.train.AdamOptimizer(learning_rate=0.001)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = opt.minimize(loss=loss, global_step=global_step)

In [63]:
with tf.variable_scope('evaluation'):
    acc, acc_op = tf.metrics.accuracy(labels=Y, predictions=predictions, name='accuracy')
    loss_summary = tf.summary.scalar('loss', loss)
    acc_summary = tf.summary.scalar('accuracy', acc)
    eval_summaries = tf.summary.merge([loss_summary, acc_summary])

In [64]:
with tf.variable_scope('cnn_model/conv_layer_1', reuse=True):
    kernel = tf.get_variable('kernel')
    with tf.variable_scope('visualization'):
        # scale weights to [0 1]
        x_min = tf.reduce_min(kernel)
        x_max = tf.reduce_max(kernel)
        kernel_0_to_1 = (kernel - x_min) / (x_max - x_min)

        # to tf.summary.image format
        kernel_transposed = tf.transpose(kernel_0_to_1, [3, 0, 1, 2])

        # this will display 5 filters from the 8 in conv_layer_1
        filter_summary = tf.summary.image('filters', kernel_transposed, max_outputs=5)

In [65]:
LOG_DIR = os.path.join(os.curdir, 'parkingSign_log')
NUM_EPOCHS = 1000
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
# with tf.Session() as sess:
sess=tf.Session()
sess.run(init)
# Instantiate writers for TensorBoard (for saving serialized summaries to disk)
train_summary_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'), sess.graph)
test_summary_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'), sess.graph)
# Run optimizer for multiple epochs
for epoch in range(NUM_EPOCHS):
    print("Starting epoch {}.".format(epoch))
    for X_batch, Y_batch in train_iterator:
        # Run a training step
        _, step = sess.run([train_op, global_step],
                           feed_dict={X: X_batch, Y: Y_batch, training_mode: True})
        # Every 100 batches compute the accuracy on the training set and save the filters in the first convolutional layer
        if (step % 100 == 0 and step > 0):
            train_accuracy, eval_s, filter_s = sess.run([acc_op, eval_summaries, filter_summary], 
                              feed_dict={X: images28, Y: labels, training_mode: False})
            train_summary_writer.add_summary(eval_s, global_step=step)
            train_summary_writer.add_summary(filter_s, global_step=step)
            print("\tTraining accuracy at step {}: {}.".format(step, train_accuracy))
        # Every 10 batches compute the accuracy on the test set.
        if (step % 10 == 0):
            test_accuracy, eval_s = sess.run([acc_op, eval_summaries], 
                             feed_dict={X: test_images28, Y: test_labels, training_mode: False})
            test_summary_writer.add_summary(eval_s, global_step=step)
print("Optimization complete.")
train_summary_writer.close()
test_summary_writer.close()

Starting epoch 0.
Starting epoch 1.
	Training accuracy at step 100: 0.6924149394035339.
Starting epoch 2.
	Training accuracy at step 200: 0.7506569027900696.
Starting epoch 3.
	Training accuracy at step 300: 0.7831354737281799.
Starting epoch 4.
	Training accuracy at step 400: 0.8036463260650635.
Starting epoch 5.
	Training accuracy at step 500: 0.8192464113235474.
Starting epoch 6.
	Training accuracy at step 600: 0.8329309225082397.
Starting epoch 7.
Starting epoch 8.
	Training accuracy at step 700: 0.8422640562057495.
Starting epoch 9.
	Training accuracy at step 800: 0.8513946533203125.
Starting epoch 10.
	Training accuracy at step 900: 0.8586626052856445.
Starting epoch 11.
	Training accuracy at step 1000: 0.8652713298797607.
Starting epoch 12.
	Training accuracy at step 1100: 0.8712807893753052.
Starting epoch 13.
	Training accuracy at step 1200: 0.8758790493011475.
Starting epoch 14.
	Training accuracy at step 1300: 0.8798774480819702.
Starting epoch 15.
Starting epoch 16.
	Traini

	Training accuracy at step 11000: 0.9600799083709717.
Starting epoch 127.
	Training accuracy at step 11100: 0.9602503180503845.
Starting epoch 128.
	Training accuracy at step 11200: 0.9604136943817139.
Starting epoch 129.
	Training accuracy at step 11300: 0.9605722427368164.
Starting epoch 130.
Starting epoch 131.
	Training accuracy at step 11400: 0.9604904651641846.
Starting epoch 132.
	Training accuracy at step 11500: 0.9604196548461914.
Starting epoch 133.
	Training accuracy at step 11600: 0.9605066776275635.
Starting epoch 134.
	Training accuracy at step 11700: 0.9606326818466187.
Starting epoch 135.
	Training accuracy at step 11800: 0.9607844948768616.
Starting epoch 136.
	Training accuracy at step 11900: 0.9609485864639282.
Starting epoch 137.
	Training accuracy at step 12000: 0.9611172676086426.
Starting epoch 138.
Starting epoch 139.
	Training accuracy at step 12100: 0.9612663388252258.
Starting epoch 140.
	Training accuracy at step 12200: 0.9614201784133911.
Starting epoch 141

Starting epoch 249.
	Training accuracy at step 21700: 0.9686729907989502.
Starting epoch 250.
	Training accuracy at step 21800: 0.9687137007713318.
Starting epoch 251.
	Training accuracy at step 21900: 0.9687638878822327.
Starting epoch 252.
	Training accuracy at step 22000: 0.9688125848770142.
Starting epoch 253.
Starting epoch 254.
	Training accuracy at step 22100: 0.9688685536384583.
Starting epoch 255.
	Training accuracy at step 22200: 0.968912661075592.
Starting epoch 256.
	Training accuracy at step 22300: 0.9689667224884033.
Starting epoch 257.
	Training accuracy at step 22400: 0.9690183401107788.
Starting epoch 258.
	Training accuracy at step 22500: 0.9690650701522827.
Starting epoch 259.
	Training accuracy at step 22600: 0.9691091775894165.
Starting epoch 260.
	Training accuracy at step 22700: 0.9691492915153503.
Starting epoch 261.
Starting epoch 262.
	Training accuracy at step 22800: 0.9691904783248901.
Starting epoch 263.
	Training accuracy at step 22900: 0.9692402482032776.

Starting epoch 372.
	Training accuracy at step 32400: 0.9723559617996216.
Starting epoch 373.
	Training accuracy at step 32500: 0.972383439540863.
Starting epoch 374.
	Training accuracy at step 32600: 0.9724121689796448.
Starting epoch 375.
	Training accuracy at step 32700: 0.9724372625350952.
Starting epoch 376.
Starting epoch 377.
	Training accuracy at step 32800: 0.9724637866020203.
Starting epoch 378.
	Training accuracy at step 32900: 0.9724812507629395.
Starting epoch 379.
	Training accuracy at step 33000: 0.972503662109375.
Starting epoch 380.
	Training accuracy at step 33100: 0.9725320339202881.
Starting epoch 381.
	Training accuracy at step 33200: 0.9725587368011475.
Starting epoch 382.
	Training accuracy at step 33300: 0.972588300704956.
Starting epoch 383.
	Training accuracy at step 33400: 0.9726184606552124.
Starting epoch 384.
Starting epoch 385.
	Training accuracy at step 33500: 0.9726409316062927.
Starting epoch 386.
	Training accuracy at step 33600: 0.9726448655128479.
S

Starting epoch 495.
	Training accuracy at step 43100: 0.9742451310157776.
Starting epoch 496.
	Training accuracy at step 43200: 0.97426438331604.
Starting epoch 497.
	Training accuracy at step 43300: 0.9742803573608398.
Starting epoch 498.
	Training accuracy at step 43400: 0.9742974042892456.
Starting epoch 499.
	Training accuracy at step 43500: 0.9743074178695679.
Starting epoch 500.
Starting epoch 501.
	Training accuracy at step 43600: 0.9743192791938782.
Starting epoch 502.
	Training accuracy at step 43700: 0.9743312001228333.
Starting epoch 503.
	Training accuracy at step 43800: 0.9743483662605286.
Starting epoch 504.
	Training accuracy at step 43900: 0.9743590354919434.
Starting epoch 505.
	Training accuracy at step 44000: 0.97437584400177.
Starting epoch 506.
	Training accuracy at step 44100: 0.9743926525115967.
Starting epoch 507.
Starting epoch 508.
	Training accuracy at step 44200: 0.9744073748588562.
Starting epoch 509.
	Training accuracy at step 44300: 0.9744205474853516.
St

Starting epoch 618.
	Training accuracy at step 53800: 0.9752994775772095.
Starting epoch 619.
	Training accuracy at step 53900: 0.9753066897392273.
Starting epoch 620.
	Training accuracy at step 54000: 0.9753120541572571.
Starting epoch 621.
	Training accuracy at step 54100: 0.9753198623657227.
Starting epoch 622.
	Training accuracy at step 54200: 0.9753291606903076.
Starting epoch 623.
Starting epoch 624.
	Training accuracy at step 54300: 0.9753360152244568.
Starting epoch 625.
	Training accuracy at step 54400: 0.9753414988517761.
Starting epoch 626.
	Training accuracy at step 54500: 0.9753470420837402.
Starting epoch 627.
	Training accuracy at step 54600: 0.9753502011299133.
Starting epoch 628.
	Training accuracy at step 54700: 0.9753499031066895.
Starting epoch 629.
	Training accuracy at step 54800: 0.9753563404083252.
Starting epoch 630.
Starting epoch 631.
	Training accuracy at step 54900: 0.9753596186637878.
Starting epoch 632.
	Training accuracy at step 55000: 0.9753667116165161

Starting epoch 741.
	Training accuracy at step 64500: 0.9758659601211548.
Starting epoch 742.
	Training accuracy at step 64600: 0.9758735299110413.
Starting epoch 743.
	Training accuracy at step 64700: 0.9758757948875427.
Starting epoch 744.
	Training accuracy at step 64800: 0.9758771657943726.
Starting epoch 745.
	Training accuracy at step 64900: 0.9758684635162354.
Starting epoch 746.
Starting epoch 747.
	Training accuracy at step 65000: 0.9758694767951965.
Starting epoch 748.
	Training accuracy at step 65100: 0.9758728742599487.
Starting epoch 749.
	Training accuracy at step 65200: 0.9758772253990173.
Starting epoch 750.
	Training accuracy at step 65300: 0.9758808016777039.
Starting epoch 751.
	Training accuracy at step 65400: 0.9758831858634949.
Starting epoch 752.
	Training accuracy at step 65500: 0.9758867025375366.
Starting epoch 753.
Starting epoch 754.
	Training accuracy at step 65600: 0.9758870601654053.
Starting epoch 755.
	Training accuracy at step 65700: 0.9758889079093933

Starting epoch 864.
	Training accuracy at step 75200: 0.9762555956840515.
Starting epoch 865.
	Training accuracy at step 75300: 0.9762591123580933.
Starting epoch 866.
	Training accuracy at step 75400: 0.9762611985206604.
Starting epoch 867.
	Training accuracy at step 75500: 0.9762645363807678.
Starting epoch 868.
	Training accuracy at step 75600: 0.9762688279151917.
Starting epoch 869.
Starting epoch 870.
	Training accuracy at step 75700: 0.9762741923332214.
Starting epoch 871.
	Training accuracy at step 75800: 0.9762810468673706.
Starting epoch 872.
	Training accuracy at step 75900: 0.9762844443321228.
Starting epoch 873.
	Training accuracy at step 76000: 0.9762889742851257.
Starting epoch 874.
	Training accuracy at step 76100: 0.9762954115867615.
Starting epoch 875.
	Training accuracy at step 76200: 0.9762967228889465.
Starting epoch 876.
Starting epoch 877.
	Training accuracy at step 76300: 0.9762957096099854.
Starting epoch 878.
	Training accuracy at step 76400: 0.9762963056564331

Starting epoch 987.
	Training accuracy at step 85900: 0.9765476584434509.
Starting epoch 988.
	Training accuracy at step 86000: 0.9765486717224121.
Starting epoch 989.
	Training accuracy at step 86100: 0.9765501618385315.
Starting epoch 990.
	Training accuracy at step 86200: 0.9765542149543762.
Starting epoch 991.
	Training accuracy at step 86300: 0.9765554070472717.
Starting epoch 992.
Starting epoch 993.
	Training accuracy at step 86400: 0.976556122303009.
Starting epoch 994.
	Training accuracy at step 86500: 0.9765576124191284.
Starting epoch 995.
	Training accuracy at step 86600: 0.976561427116394.
Starting epoch 996.
	Training accuracy at step 86700: 0.9765663146972656.
Starting epoch 997.
	Training accuracy at step 86800: 0.9765684008598328.
Starting epoch 998.
	Training accuracy at step 86900: 0.9765722155570984.
Starting epoch 999.
	Training accuracy at step 87000: 0.9765766859054565.
Optimization complete.


In [66]:
# Run predictions against the full test set.

predicted = sess.run([predictions], feed_dict={X: test_images28, training_mode: False})[0]
print(predicted)
print(len(predicted))
# # Calculate correct matches
# match_count = sum([int(y == y_) for y, y_ in zip(test_labels, predicted)])

# # Calculate the accuracy
# accuracy = match_count / len(test_labels)

# # Print the accuracy
# print("Accuracy: {:.3f}".format(accuracy))

[0 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1 1 0 0 1 0 1 1 0
 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 0 1
 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 0 1
 1 0 1 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1
 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0
 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1
 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1 1 1
 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0
 1 1 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1
 1 0 1 0 1 0 1 1 0 0 1 1 

In [67]:
np.sum(predicted)

449

In [68]:
def precision(labels, predicted):
    FP=0
    TP=0
    for i in range(len(labels)):
        if labels[i]=='1' and predicted[i]==1:
            TP+=1
        if labels[i]=='0' and predicted[i]==1:
            FP+=1
    print(TP)
    print(FP)
    return TP/(TP+FP)


In [69]:
def recall(labels, predicted):
    FN=0
    TP=0
    for i in range(len(labels)):
        if labels[i]=='1' and predicted[i]==1:
            TP+=1
        if labels[i]=='1' and predicted[i]==0:
            FN+=1
    print("FN:")
    print(FN)
    return TP/(TP+FN)

In [70]:
# Calculate correct matches
match_count = sum([int(int(y) == y_) for y, y_ in zip(test_labels, predicted)])
print(match_count)
# Calculate the accuracy
accuracy = match_count / len(test_labels)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))
# Print the recall and precision
print("Recall: {:3f}".format(recall(test_labels, predicted)))
print("Precision: {:3f}".format(precision(test_labels, predicted)))

919
Accuracy: 0.960
FN:
5
Recall: 0.988124
416
33
Precision: 0.926503


In [75]:
#Load output
output_data_directory="/Users/chenlingna/Desktop/2ndTrial/Negative"
output_images, output_labels, output_names = load_data(output_data_directory)

output_images28=[]
for img in output_images:
#     img[img>color]=1
#     img[img<color]=0
    output_images28.append(img)

# Transform the images to 28 by 28 pixels
output_images28 = [transform.resize(image, (28, 28)) for image in output_images28]
output_images28 = np.array(output_images28)

# Run predictions against the output set.
predicted = sess.run([predictions], feed_dict={X: output_images28, training_mode: False})[0]


# Calculate correct matches
match_count = sum([int(int(y) == y_) for y, y_ in zip(output_labels, predicted)])
# Calculate the accuracy
accuracy = match_count / len(output_labels)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))
# Print the recall and precision
print("Recall: {:3f}".format(recall(output_labels, predicted)))
print("Precision: {:3f}".format(precision(output_labels, predicted)))

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Accuracy: 0.975
FN:
0


ZeroDivisionError: division by zero

In [76]:
np.sum(predicted)

498

In [78]:
#output the images
import scipy.misc
for i in range(len(output_images28)):
    if predicted[i]==0:
        name=output_names[i].split('/')[-1]
        scipy.misc.imsave("/Users/chenlingna/Desktop/output-F/Negative/"+name, skimage.data.imread(output_names[i]))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """
